# Model Architecture

          [ Question Text ]               [ Candidate Demos ]
                  ↓                                ↓
            MathBERT Encoder               MathBERT Encoder
                  ↓                                ↓
             Q_embedding                      {C_embeddings}

           [Selected Demos]
                  ↓
             MathBERT Encoder
                  ↓
          {Sel_embeddings}

                  ↓
     Context Aggregation (mean pooling or attention)

                  ↓
    For each candidate:
    [Context || Candidate_emb] → MLP Scorer → Scalar Score

                  ↓
       Softmax over candidate scores
                  ↓
      Select next demo (sample/argmax)
                  ↓
  Add to selected demos, repeat until k selected

                  ↓
    Feed (k demos + question) to ICL model
                  ↓
           Generate answer
                  ↓
    Reward based on correctness
                  ↓
         PPO/GRPO optimization


# Testing the MathBERT Encoder for [CLS] token

In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("tbs17/MathBERT")

text = "Find the value of x in 2x + 3 = 7"
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

encoded = tokenizer(text, return_tensors="pt")
input_ids = encoded["input_ids"][0]

# Convert token IDs back to tokens to see full sequence
tokens_with_special = tokenizer.convert_ids_to_tokens(input_ids)
print("Tokenized with special tokens:", tokens_with_special)


c:\Users\abhin\miniconda3\envs\lima\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abhin\miniconda3\envs\lima\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhin\.cache\huggingface\hub\models--tbs17--MathBERT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer m

Tokens: ['find', 'the', 'value', 'of', 'x', 'in', '2', '##x', '+', '3', '=', '7']
Tokenized with special tokens: ['[CLS]', 'find', 'the', 'value', 'of', 'x', 'in', '2', '##x', '+', '3', '=', '7', '[SEP]']


# Testing the Retriever Pipeline

In [3]:
import torch
from trainer.retriever_pipeline import RetrieverPipeline

# Initialize the pipeline
pipeline = RetrieverPipeline(device="cuda")  # or "cpu" if no GPU

# Define an inference question
Q_inf = "What is the value of 3 + 7?"

# Define candidate demos (Q, A) pairs
demos = [
    ("What is 2+2?", "4"),
    ("What is 5x5?", "25"),
    ("What is 10/2?", "5"),
    ("What is 7-3?", "4"),
]

# Set number of demos to retrieve
k = 2  # 2-shot

# Retrieve and build prompt
prompt, selected_indices = pipeline.retrieve_and_build_prompt(
    Q_inf=Q_inf,
    demos=demos,
    k=k,
    sample=False  # use greedy selection
)

# Print outputs
print("\n=== Selected Demo Indices ===")
print(selected_indices)

print("\n=== Generated ICL Prompt ===")
print(prompt)


=== Selected Demo Indices ===
[1, 0]

=== Generated ICL Prompt ===
Q: What is 5x5?
A: 25

Q: What is 2+2?
A: 4

Q: What is the value of 3 + 7?
A:
